In [1]:
import requests as re
import pandas as pd
import numpy as np
import glob
import os
from bs4 import BeautifulSoup

import time

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)



In [3]:
def getPosts(thread_url):
    lastThreadPage = False
    pageCount = 0
    thread_cols = ['User_id','Type','Content_id','Url','Reply_to_content_id','Thread_id','Subforum','Title','Content','Timestamp', 'Search_Term'] 
    thread_df = pd.DataFrame(columns=thread_cols, dtype=object)
    thread_page_url = thread_url[0]
    search_term = thread_url[1]
    Thread_ID = ''
    title = ''
    userid = ''
    post_id = ''
    subforum = ''
    iso_datetime = ''
#     print("Searching for root url: " + thread_url[0])

    while(not lastThreadPage and pageCount < 1000):
        try:
            r3 = re.get(thread_page_url)
            pageCount += 1
            thread_page = r3.text
            thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

            if (thread_page_soup.find('a', text='Next') == None):
                lastThreadPage = True
            else:
                #try:
                thread_page_url = site_url + thread_page_soup.find('a', text='Next')['href']
#                 print(thread_page_url)  
                Thread_ID = thread_page_soup.find('html')['data-content-key'] .strip('thread-') 
                title = thread_page_soup.find('h1',{'class':'p-title-value'} ).text
                #except:
                    #pass

            for element in thread_page_soup.find_all('div',{"class": "p-body-inner"}): 
                all_x_tag = element.find_all('a')
                subforum = all_x_tag[2].text.replace('\n','').replace('\r','')        

                for post in element.find_all('div',{"class": "message-inner"}): 
                    all_a_tag = post.find_all('a')


                    userid_url = site_url + all_a_tag[0]['href']

                    try:
                        userid = all_a_tag[0]['data-user-id']
                    except KeyError:
                        pass



                    all_time_tag = post.find_all('time')
                    all_b_tag = post.find_all('a')
                    iso_datetime0 = all_time_tag[0]['datetime']
                    iso_datetime = iso_datetime0[:-5]
                    #print(iso_datetime)
                    post_text = ""

                    post_id = [tag['data-lb-id'].lstrip('post-') for tag in post.select('div[data-lb-id]')] 


                    first_quote_post_number = np.nan            

                    for text in post.find_all('div',{"class": "bbWrapper"}):
                        post_text = text.get_text(' ', strip=True)
                        #print(post_text)
                        try:
                            first_quote = text.find('div', {'class': 'bbCodeBlock-title'})
                            first_quote_post_number = first_quote.find('a')['data-content-selector'].split('#')[1].lstrip('post-')
                        except:
                                pass

                    row = pd.DataFrame([[userid, 'Post', post_id,thread_url[0], first_quote_post_number, Thread_ID, subforum, title, post_text,iso_datetime, thread_url[1]]], columns=thread_cols)
                    if(len(thread_df)==0):
                        thread_df = row
                    else:
                        thread_df = thread_df.concat(row, ignore_index=True) #df.append doesn't work inplace
        except:
            pass

    thread_df['Content'] = thread_df['Content'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))
    print(time.ctime() + " Finished Searching for root url: " + thread_url[0] + " Total pages " + str(pageCount) + " Remaining " + str(executor._work_queue.qsize()))

    return thread_df

In [4]:
site_url = 'https://forums.hardwarezone.com.sg'
url_df_1 = pd.read_csv("hardware_zone_raw_data_without_detail.csv")
Thread_csv_URL = list(zip(url_df_1.URL, url_df_1.Search_Term))
# print(Thread_csv_URL)

root_url = set()
for thread in Thread_csv_URL:
    if 'page-' in thread[0]:
        position = thread[0].rfind('/page')
        root_url.add((thread[0][:position],thread[1]))
    else:
        root_url.add(thread)        
        

In [5]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

startTime = time.time()
executor = ThreadPoolExecutor(max_workers=200)
result_list = []

# for url in root_url:
#     result_list.append(getPosts(url))

for result in executor.map(getPosts, root_url):
    result_list.append(result)

endTime = time.time()
print(str(endTime-startTime))
post_df_1 = pd.concat(result_list ,ignore_index=True)
post_df_1.to_csv("hardware_zone_raw_data_with_detail_multithread.csv")

Tue Apr  4 03:13:30 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/bachelors-second-upper-class-honours.5861595/ Total pages 1 Remaining 1
Tue Apr  4 03:13:31 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/need-advice-home-equity-loan.6880367/ Total pages 1 Remaining 1
Tue Apr  4 03:13:31 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/news-high-court-rules-that-accident-victim-can-claim-govt-medical-subsidies-after-vehicle-accident-but-has-to-return-full-sum-to-moh.6884961/ Total pages 1 Remaining 0
Tue Apr  4 03:13:33 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/sg-will-be-great-again-if-tt-ceo-become-sg-pm.6886140/ Total pages 1 Remaining 1
Tue Apr  4 03:13:35 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/st-new-hawker-centre-is-first-such-facility-in-sembawang-in-30-years.6849693/ Total pages 1 Remaining 1
Tue Ap

Tue Apr  4 03:14:13 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/why-diploma-and-degree-courses-cost-so-much.6886584/ Total pages 2 Remaining 0
Tue Apr  4 03:14:14 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/renting-out-property-without-an-agent.5317694 Total pages 3 Remaining 1
Tue Apr  4 03:14:16 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/how-long-to-take-class-4.5718422 Total pages 2 Remaining 0
Tue Apr  4 03:14:16 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/what-make-a-good-wife.6887236 Total pages 3 Remaining 0
Tue Apr  4 03:14:18 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/what-is-middle-class-upper-middle-class-income.6809953/ Total pages 5 Remaining 1
Tue Apr  4 03:14:19 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/which-private-dental-clinic-is-cheap-and-good.

Tue Apr  4 03:15:01 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/glgt-cpf-salary-ceiling-should-keep-pace-with-inflation-to-avoid-problems-for-singapore-in-the-future-lawrence-wong.6873894 Total pages 4 Remaining 0
Tue Apr  4 03:15:01 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/my-colleague-is-depressed-because-no-insurer-will-cover-him-for-hospitalisation.6886643 Total pages 3 Remaining 8
Tue Apr  4 03:15:01 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/price-comparing-for-lasting-power-of-attorney.5430297 Total pages 7 Remaining 7
Tue Apr  4 03:15:04 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/how-easy-is-it-to-get-a-fake-singapore-university-degree-certificate-and-what-are-firms-doing-to-detect-this.6882854 Total pages 4 Remaining 0
Tue Apr  4 03:15:05 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/can-anyo

Tue Apr  4 03:15:50 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/bto-rules-tightening-home-buyers-worry-they-may-be-forced-to-choose-a-bad-flat.6878686/ Total pages 7 Remaining 824
Tue Apr  4 03:15:55 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/covid-19-treatment-subsidies-to-be-scaled-back-as-singapore-progressively-returns-to-normal-moh.6762933/ Total pages 2 Remaining 823
Tue Apr  4 03:15:58 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/ Total pages 8 Remaining 822
Tue Apr  4 03:16:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/singapore-fertility-rate-hits-new-low-putting-focus-on-housing-prices.6876887 Total pages 5 Remaining 821Tue Apr  4 03:16:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/how-much-do-you-budget-for-your-parents-healthcare-needs.6874633

Tue Apr  4 03:16:58 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/mr-chew-does-tiktok-access-the-home-wifi-network-were.6887260/ Total pages 2 Remaining 782
Tue Apr  4 03:17:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/how-to-define-middle-income-group.6694447/ Total pages 1 Remaining 781Tue Apr  4 03:17:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/dehumidifier.6879510 Total pages 5 Remaining 781
Tue Apr  4 03:17:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/fake-degrees-saga.6882897 Total pages 5 Remaining 780

Tue Apr  4 03:17:01 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/desmond-lee-says-singaporeans-accept-high-price-bto-flats-in-mature-estates.6887271 Total pages 8 Remaining 778
Tue Apr  4 03:17:02 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/household-income-

...

Tue Apr  4 03:54:28 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/why-diploma-and-degree-courses-cost-so-much.6886584 Total pages 2 Remaining 22
Tue Apr  4 03:54:44 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/actually-whats-your-reason-for-staying-in-sg.6875736 Total pages 6 Remaining 21
Tue Apr  4 03:54:58 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/starhill-global-reit-official-sgx-p40u.2766554 Total pages 65 Remaining 20
Tue Apr  4 03:55:05 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/official-november-2022-bto.6755776/ Total pages 48 Remaining 19
Tue Apr  4 03:55:17 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/is-it-wise-to-buy-a-car-in-singapore.6873339 Total pages 6 Remaining 18
Tue Apr  4 03:55:17 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/do-gov-subsidy-medishiel


Tue Apr  4 03:59:34 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/income-tax-filing-2023.6879395 Total pages 10 Remaining 0
Tue Apr  4 03:59:46 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/for-couples-with-low-income.6584995 Total pages 27 Remaining 0Tue Apr  4 03:59:47 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/endowus-roboadvisor-investing-using-cpf.6151140 Total pages 59 Remaining 0

Tue Apr  4 04:00:06 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/official-november-2022-bto.6755776/ Total pages 48 Remaining 0
Tue Apr  4 04:00:24 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/many-hawkers-retiring-due-to-manpower-shortage.6886842 Total pages 6 Remaining 0
Tue Apr  4 04:00:25 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/which-mba-degree-most-bang-for-buck.6881271 Total pag

Tue Apr  4 04:26:58 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/hsbc-visa-infinite-card.3438220 Total pages 105 Remaining 0
Tue Apr  4 04:30:50 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/hdb-resale-flat-time-line-discussion.6362932 Total pages 95 Remaining 0
Tue Apr  4 04:32:04 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/free-advice-discussion-on-buy-sell-of-resale-flat-part-6-part-2.6361914 Total pages 134 Remaining 0
Tue Apr  4 04:33:07 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/anyone-here-have-plans-to-migrate.6690835 Total pages 107 Remaining 0
Tue Apr  4 04:33:23 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/debt-consolidation-plan-part-2.6035250 Total pages 112 Remaining 0
Tue Apr  4 04:34:02 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/free-advice-discussion-on

Tue Apr  4 10:35:11 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 Total pages 299 Remaining 0
Tue Apr  4 10:37:02 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 Total pages 299 Remaining 0
Tue Apr  4 10:38:03 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 Total pages 299 Remaining 0
Tue Apr  4 10:41:06 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 Total pages 299 Remaining 0
Tue Apr  4 10:42:32 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 Total pages 299 Remaining 0
Tue Apr  4 10:43:14 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167 To

Tue Apr  4 12:42:20 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/lastest-s-deposit-updates-part-3.5575909 Total pages 739 Remaining 0
Tue Apr  4 12:43:12 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/official-edmw-jobless-club-part-8.6354724 Total pages 691 Remaining 0
Tue Apr  4 12:43:41 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/which-direction-will-property-prices-go.6382009 Total pages 748 Remaining 0
Tue Apr  4 12:44:00 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/which-direction-will-property-prices-go.6382009 Total pages 748 Remaining 0
Tue Apr  4 12:44:01 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/which-direction-will-property-prices-go.6382009 Total pages 748 Remaining 0
Tue Apr  4 12:44:30 2023 Finished Searching for root url: https://forums.hardwarezone.com.sg/threads/lastest-s-deposit-updates-p